In [14]:
#firstly, extracting all items inside a user's iventory (only cs2 items)
import requests
import time
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/117.0.0.0 Safari/537.36"
}
def fetch_inventory_page(steamid64: str, appid: int, contextid: int, start: str = None):
    url = f"https://steamcommunity.com/inventory/{steamid64}/{appid}/{contextid}"
    resp = requests.get(url,headers=HEADERS, timeout=15)
    resp.raise_for_status()
    return resp.json()
def inventory_items_for_user(steamid64: str, appid: int, contextid: int, delay_between_requests: float = 0.5):
    items = []
    seen_assetids = set()
    start = None

    while True:
        data = fetch_inventory_page(steamid64, appid, contextid, start)
        # check if inventory is private or empty
        if not data or "assets" not in data or "descriptions" not in data:
            # Might be private or zero items; return whatever we have or raise
            return items, data

        assets = data.get("assets", [])
        descriptions = data.get("descriptions", [])

        # Build lookup for descriptions by (classid, instanceid)
        desc_lookup = {}
        for d in descriptions:
            key = (str(d.get("classid")), str(d.get("instanceid", "0")))
            desc_lookup[key] = d

        for a in assets:
            assetid = a.get("assetid")  # unique per item
            if assetid in seen_assetids:
                continue
            seen_assetids.add(assetid)

            classid = str(a.get("classid"))
            instanceid = str(a.get("instanceid", "0"))
            desc = desc_lookup.get((classid, instanceid)) or {}

            item = {
                "assetid": assetid,
                "classid": classid,
                "instanceid": instanceid,
                "amount": a.get("amount", 1),
                "market_tradable_restriction": a.get("market_tradable_restriction"),
                # descriptive fields:
                "name": desc.get("name"),
                "type": desc.get("type"),
                "market_hash_name": desc.get("market_hash_name"),
                "tradable": desc.get("tradable"),
                "tags": desc.get("tags", []),
                "marketable": desc.get("marketable"),
                # entire desc for debugging
                "description_raw": desc
            }
            items.append(item)

        # pagination
        more = data.get("more_items") or False
        # some responses give 'last_assetid' to use as start; others use 'more_start'
        if more:
            start = data.get("last_assetid") or data.get("more_start")
            if not start:
                # fallback: if server indicates 'more_items' but no token, stop to avoid infinite loop
                break
            time.sleep(delay_between_requests)  # be polite
        else:
            break

    return items, data

In [15]:
# Replace these with real values
steamid64 = "76561198923974658"   # example SteamID64
appid = 730                       # likely appid for CS:GO/CS2 (confirm)
contextid = 2                     # common contextid; confirm for your target game

items, raw = inventory_items_for_user(steamid64, appid, contextid)
print(f"Found {len(items)} items")
for i, it in enumerate(items[:200], start=1):
    print(f"{i}. {it['name']!r} {it['market_hash_name']!r} ({it['assetid']}) - tradable={it.get('tradable')} marketable={it.get('marketable')}")

Found 48 items
1. 'Sticker | Aleksib (Gold) | Copenhagen 2024' 'Sticker | Aleksib (Gold) | Copenhagen 2024' (47386508166) - tradable=1 marketable=1
2. '2025 Service Medal' '2025 Service Medal' (47380657304) - tradable=0 marketable=0
3. 'Revolution Case' 'Revolution Case' (47341167845) - tradable=1 marketable=1
4. 'Revolution Case' 'Revolution Case' (47185843227) - tradable=1 marketable=1
5. 'MP5-SD | Dirt Drop' 'MP5-SD | Dirt Drop (Field-Tested)' (47017281017) - tradable=1 marketable=1
6. 'MP9 | Buff Blue' 'MP9 | Buff Blue (Battle-Scarred)' (46861180799) - tradable=1 marketable=1
7. 'Sticker | Nemiga (Foil) | Austin 2025' 'Sticker | Nemiga (Foil) | Austin 2025' (46772516572) - tradable=1 marketable=1
8. 'Sticker | broky (Foil) | Austin 2025' 'Sticker | broky (Foil) | Austin 2025' (46497303228) - tradable=1 marketable=1
9. 'Sticker | m0NESY (Foil) | Austin 2025' 'Sticker | m0NESY (Foil) | Austin 2025' (46497275563) - tradable=1 marketable=1
10. 'Sticker | Heavygod (Gold) | Austin 2025' 

In [16]:
item_summaries = []

for i, it in enumerate(items[:200], start=1):
    summary = {
        "index": i,
        "name": it['name'],
        "market_hash_name": it['market_hash_name'],
        "tradable": it.get('tradable'),
        "marketable": it.get('marketable')
    }
    item_summaries.append(summary)

# use item_summaries as a variable
print(item_summaries)

[{'index': 1, 'name': 'Sticker | Aleksib (Gold) | Copenhagen 2024', 'market_hash_name': 'Sticker | Aleksib (Gold) | Copenhagen 2024', 'tradable': 1, 'marketable': 1}, {'index': 2, 'name': '2025 Service Medal', 'market_hash_name': '2025 Service Medal', 'tradable': 0, 'marketable': 0}, {'index': 3, 'name': 'Revolution Case', 'market_hash_name': 'Revolution Case', 'tradable': 1, 'marketable': 1}, {'index': 4, 'name': 'Revolution Case', 'market_hash_name': 'Revolution Case', 'tradable': 1, 'marketable': 1}, {'index': 5, 'name': 'MP5-SD | Dirt Drop', 'market_hash_name': 'MP5-SD | Dirt Drop (Field-Tested)', 'tradable': 1, 'marketable': 1}, {'index': 6, 'name': 'MP9 | Buff Blue', 'market_hash_name': 'MP9 | Buff Blue (Battle-Scarred)', 'tradable': 1, 'marketable': 1}, {'index': 7, 'name': 'Sticker | Nemiga (Foil) | Austin 2025', 'market_hash_name': 'Sticker | Nemiga (Foil) | Austin 2025', 'tradable': 1, 'marketable': 1}, {'index': 8, 'name': 'Sticker | broky (Foil) | Austin 2025', 'market_hash

In [ ]:
#now, use the name of items, and fetch the current market price
#Note: we do need cookies for this request.
from urllib.parse import quote
def get_market_price(market_hash_name, appid=730, currency=1):
    base_url = "http://steamcommunity.com/market/priceoverview/"
    params = {
        "appid": appid,
        "currency": currency,
        "market_hash_name": market_hash_name
    }
    url = base_url + f"?appid={appid}&currency={currency}&market_hash_name=" + quote(market_hash_name)
    #We need the cookies input manually, or getting the cookies on a logged on browser onto steam automatically
    # cookie_str = ""

    # cookies = dict(item.split("=", 1) for item in cookie_str.split("; "))
    # resp = requests.get(url,headers=HEADERS,cookies=cookies)
    #update: we wont need cookies, just change the url to http, instead of https.
    resp = requests.get(url,headers=HEADERS)
    data = resp.json()

    if data.get("success"):
        return {
            "lowest_price": data.get("lowest_price"),
            "median_price": data.get("median_price"),
            "volume": data.get("volume")
        }
    else:
        return None

In [ ]:
import time
import random
for item in item_summaries:
    name = item["market_hash_name"]
    price_info = get_market_price(name)
    
    if price_info:
        print(f"{name}: {price_info['lowest_price']} (median: {price_info['median_price']})")
    else:
        print(f"{name}: Price not found")
    #adding delays for bypass steam's rate limiting
    time.sleep(random.uniform(4.5, 6.0))

Sticker | Aleksib (Gold) | Copenhagen 2024: $4.33 (median: $4.34)
2025 Service Medal: None (median: None)
Revolution Case: $0.55 (median: $0.58)
Revolution Case: $0.55 (median: $0.58)
MP5-SD | Dirt Drop (Field-Tested): $0.06 (median: $0.04)
MP9 | Buff Blue (Battle-Scarred): $0.06 (median: $0.03)
Sticker | Nemiga (Foil) | Austin 2025: $0.24 (median: $0.23)
Sticker | broky (Foil) | Austin 2025: $0.27 (median: $0.28)
Sticker | m0NESY (Foil) | Austin 2025: $2.42 (median: $2.29)
Sticker | Heavygod (Gold) | Austin 2025: $2.73 (median: $2.60)
Sticker | m0NESY | Austin 2025: $0.24 (median: $0.21)
Sticker | broky (Foil) | Austin 2025: $0.27 (median: $0.28)
Sticker | TYLOO | Austin 2025: $0.18 (median: $0.19)
Sticker | EliGE (Foil) | Austin 2025: $0.27 (median: $0.25)
Sticker | MOUZ (Glitter) | Shanghai 2024: $0.51 (median: $0.53)
Sticker | Wildcard (Foil) | Austin 2025: $0.19 (median: $0.19)
Sticker | magixx (Holo) | Austin 2025: $2.80 (median: $2.78)
Sticker | m0NESY | Austin 2025: $0.24 (medi